## Importações e bibliotecas

In [ ]:
!pip install langchain openai chromadb langchainhub bs4 youtube-transcript-api tiktoken langchain-community requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.0/974.0 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.7/314.7 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
import requests
from typing import List, Dict
from langchain.document_loaders import YoutubeLoader
from langchain import hub
from langchain.chat_models import ChatOpenAI
from langchain.schema import Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import StrOutputParser
from langchain.text_splitter import HTMLHeaderTextSplitter
from langchain.vectorstores import Chroma
from langchain_core.runnables import RunnablePassthrough
import logging
import pprint

## API

In [ ]:
# Defina sua chave de API OpenAI
OPENAI_API_KEY = "Insira sua OPENIA_API_KEY"
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

## Importações de conteudos

### Importando videos do YoutTube de diferentes idiomas

In [ ]:
def load_youtube_videos(video_urls: List[Dict[str, List[str]]]) -> List[Document]:
    all_docs = []
    for video_info in video_urls:
        url = video_info["url"]
        language = video_info["language"]
        try:
            loader = YoutubeLoader.from_youtube_url(url, add_video_info=False, language=language)
            docs = loader.load()
            all_docs.extend(docs)
        except Exception as e:
            logging.error(f"Error loading YouTube video {url}: {e}")
    return all_docs

### Importando textos de diferentes idiomas

In [ ]:
def load_html_documents(urls: List[str]) -> List[Document]:
    docs_final = []
    html_splitter = HTMLHeaderTextSplitter(
        headers_to_split_on=[
            ("h1", "article_h1_main"),
            ("h2", "article_h2_subsection"),
            ("h3", "article_h3_subsection"),
            ("h4", "article_h4_subsection"),
        ],
        return_each_element=False
    )
    for url in urls:
        try:
            docs = html_splitter.split_text_from_url(url)
            for doc in docs:
                doc.metadata['source_url'] = url
            docs_final.extend(docs)
        except Exception as e:
            logging.error(f"Error accessing {url}: {e}")
    return docs_final

### Conteudos

In [ ]:
youtube_video_urls = [
    {"url": "https://youtu.be/q8dhQRjXJG4?si=3ucpT--YGU6KbQi7", "language": ["pt"]},
    {"url": "https://youtu.be/qop11qf3tTs?si=g5HDzs_kqwVRPwM_", "language": ["pt"]},
    {"url": "https://youtu.be/sygBTa3SkDI?si=_tL_zSNFRU1aumOk", "language": ["pt"]},
    {"url": "https://youtu.be/lfR6LEx6eZg?si=XmWZbVgQIz1lYctY", "language": ["pt"]},
    {"url": "https://youtu.be/nbAPLfmdrKE?si=8n2g1BWSNL07UY4-", "language": ["pt"]},
    {"url": "https://youtu.be/ibkodnWEKBE?si=FWNB7-QtVQ0vVwzg", "language": ["pt"]},
    {"url": "https://youtu.be/WAlTeEfU6as?si=94jjMiQAoP2tT707", "language": ["pt"]},
    {"url": "https://youtu.be/yc94C7jsAPM?si=xHfTWYYglAN68OOp", "language": ["pt"]},
    {"url": "https://youtu.be/Ya7wGSrZjX4?si=eSg6JgA-VRSaF8h1", "language": ["pt"]},
    {"url": "https://youtu.be/oDceh9hvPc8?si=OBDh_F8nvt-fYAAc", "language": ["pt"]},
    {"url": "https://youtu.be/odVchJ9pmgY?si=RY7Xwmdk6iYAGpfY", "language": ["pt"]},
    {"url": "https://youtu.be/knq3eOIIq08?si=j0PqKZdzBeMXOKl0", "language": ["pt"]},
    {"url": "https://youtu.be/uvikmLdK8os?si=TSiV3u_2GQpnkyPF", "language": ["pt"]},
    {"url": "https://youtu.be/RtAUxma6r6Y?si=kDgmJzpu1EACBg6r", "language": ["pt"]},
]

html_urls = [
    'https://www.gov.br/ibama/pt-br/assuntos/fiscalizacao-e-protecao-ambiental/emergencias-ambientais/acidentes-e-emergencias-ambientais',
    'https://renapsi.org.br/18-acoes-praticas-para-voce-agir-na-preservacao-do-meio-ambiente/',
    'https://www.defesacivil.pr.gov.br/servicos/Seguranca/Defesa-Civil/Saber-como-agir-em-caso-de-desastres-ambientais-0GNAl2o8',
    'https://www.teraambiental.com.br/blog-da-tera-ambiental/como-proceder-em-situacoes-de-emergencias-ambientais',
    'https://cetesb.sp.gov.br/emergencias-quimicas/tipos-de-acidentes/vazamentos-de-oleo/preparacao-para-resposta/planos-de-contingencia-para-vazamentos-de-oleo-no-mar/',
    'https://www.okena.eco.br/acidentes-ambientais/',
    'https://cetesb.sp.gov.br/wp-content/uploads/sites/22/2013/12/ac_amb_comunic_riscos.pdf',
    'https://antigo.mma.gov.br/educacao-ambiental/educa%C3%A7%C3%A3o-a-dist%C3%A2ncia/itemlist/category/115-emergencias-ambientais.html'
]

all_docs = load_youtube_videos(youtube_video_urls)
html_docs = load_html_documents(html_urls)
all_docs.extend(html_docs)

ERROR:root:Error accessing https://www.gov.br/ibama/pt-br/assuntos/fiscalizacao-e-protecao-ambiental/emergencias-ambientais/acidentes-e-emergencias-ambientais: HTTPSConnectionPool(host='www.gov.br', port=443): Max retries exceeded with url: /ibama/pt-br/assuntos/fiscalizacao-e-protecao-ambiental/emergencias-ambientais/acidentes-e-emergencias-ambientais (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1007)')))
ERROR:root:Error accessing https://www.defesacivil.pr.gov.br/servicos/Seguranca/Defesa-Civil/Saber-como-agir-em-caso-de-desastres-ambientais-0GNAl2o8: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
ERROR:root:Error accessing https://cetesb.sp.gov.br/wp-content/uploads/sites/22/2013/12/ac_amb_comunic_riscos.pdf: 'utf-8' codec can't decode byte 0xe4 in position 456: invalid continuation byte


## Modelo RAG

In [ ]:
def create_retriever(docs: List[Document]) -> Chroma:
    embeddings = OpenAIEmbeddings()
    vectorstore = Chroma.from_documents(documents=docs, embedding=embeddings)
    return vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})

def create_rag_chain(retriever) -> dict:
    prompt = hub.pull("rlm/rag-prompt")
    llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.5)

    def format_docs(docs):
        return "\n\n".join(doc.page_content for doc in docs)

    rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )
    return rag_chain

In [ ]:
retriever = create_retriever(all_docs)
rag_chain = create_rag_chain(retriever)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


## Menu de ultilização

In [ ]:
def conversation_mode():
    history = []
    while True:
        user_input = input("Você: ")
        if user_input.lower() in ["sair", "finalizar", "fim"]:
            print("Conversa finalizada.")
            break

        query_with_history = " ".join(history + [user_input])
        response = rag_chain.invoke(query_with_history)
        pp = pprint.PrettyPrinter(indent=1)
        pp.pprint(response)
        history.append(user_input)
        history.append(response)

def menu():
    while True:
        print("\nMenu:")
        print("1. Nova pergunta")
        print("2. Conversação sobre a pergunta atual")
        print("3. Finalizar o serviço")
        choice = input("Escolha uma opção: ")

        if choice == "1":
            query = input("Digite sua pergunta: ")
            response = rag_chain.invoke(query)
            pp = pprint.PrettyPrinter(indent=1)
            pp.pprint(response)
        elif choice == "2":
            print("Modo de conversação iniciado. Digite 'sair', 'finalizar' ou 'fim' para encerrar.")
            conversation_mode()
        elif choice == "3":
            print("Serviço finalizado.")
            break
        else:
            print("Opção inválida. Por favor, tente novamente.")

# Executa o menu
menu()


Menu:
1. Nova pergunta
2. Conversação sobre a pergunta atual
3. Finalizar o serviço
('Para evitar uma enchente, é importante economizar água e evitar o descarte '
 'de lixo nas ruas e estradas, que podem entupir bueiros. Além disso, ações '
 'rápidas são essenciais para minimizar os impactos, como identificar o '
 'problema, implementar medidas adequadas e isolar áreas afetadas. Medidas '
 'simples como diminuir o tempo no banho e evitar o uso excessivo de água '
 'também contribuem para a prevenção de enchentes.')

Menu:
1. Nova pergunta
2. Conversação sobre a pergunta atual
3. Finalizar o serviço
('Para proteger os animais de desastres, é essencial adotar medidas de '
 'prevenção e segurança, como planos operacionais e avaliação rápida do '
 'problema. É importante seguir as leis ambientais aplicáveis para evitar '
 'multas e garantir a conformidade. Ações rápidas e eficazes são cruciais para '
 'minimizar os impactos negativos e proteger o meio ambiente e a comunidade.')

Menu:
1. 